# RNA-Seq Data Analysis
In this jupyter notebook we will walk through some data analysis using jupyter notebooks. We will go over the general steps covered in lecture:

1) FastQ quality control
2) Read Mapping
3) Features counts
4) Simple Image generation

We will be using the data from Fu et al (2015) ' EGF- mediated induction of Mcl-1 at the switch to lactation is essential for alveolar cell survival' Nat Cell Biol.

This study examined the expression profiles of basal and luminal cells in the mammary gland of virgin, pregnant and lactating mice. Six groups are present, with one for each combination of cell type and mouse status. Note that two biological replicates are used here, two independent sorts of cells from the mammary glands of virgin, pregnant or lactating mice, however three replicates is usually recommended as a minimum requirement for RNA-seq.


As a first step we wil prepare the notebook to work with the software that we need, we will:
- Install java
- Download fastqc and make it executable

In [ ]:
!sudo apt-get install -y default-jre
!java -version

In [ ]:
!wget https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.9.zip

In [ ]:
!unzip fastqc_v0.11.9.zip

In [4]:
!ls FastQC/

cisd-jhdf5.jar	 Help		 LICENSE_JHDF5.txt  README.md	       sam-1.103.jar
Configuration	 INSTALL.txt	 LICENSE.txt	    README.txt	       Templates
fastqc		 jbzip2-0.9.jar  net		    RELEASE_NOTES.txt  uk
fastqc_icon.ico  LICENSE	 org		    run_fastqc.bat


In [5]:
!chmod +x FastQC/fastqc

In [ ]:
!./FastQC/fastqc --version

Now we will download the data from Fu et al, we will save it as example.zip (that's the -O below), we will unzip it

In [ ]:
!wget 'https://figshare.com/ndownloader/articles/3219673?private_link=f5d63d8c265a05618137' -O example.zip

In [ ]:
!unzip example.zip

Now that it is unzipped lets take a look at one (by running the code below), what do you observe, please take a moment to identify the four lines that indentify a single sequencing read.

In [ ]:
!zcat SRR1552446.fastq.gz

Now, let's take run fastqc on a single file!

In [16]:
!./FastQC/fastqc SRR1552444.fastq.gz

Started analysis of SRR1552444.fastq.gz
Approx 100% complete for SRR1552444.fastq.gz
Analysis complete for SRR1552444.fastq.gz


Based on what we covered in the lecture portion, what do you think of the data,
#### is this overall good or bad quality?

In [ ]:
#Great quality score
#Something is wrong with the beginning of each read?

Before we continue with the analysis, lets rename the files,
SRRXXXXX is the accession number in a public repository, this is how we share 'omic data generated, lets raname it based on the experimental designed presented above

|ACCESSION ID | Sample Name |
|---|---|
|SRR1552444   |  Sample_luminalvirgin_01 |
|SRR1552445   |  Sample_luminalvirgin_02 |
|SRR1552446   |  Sample_luminalpregnant_01 |
|SRR1552447   |  Sample_luminalpregnant_02 |
|SRR1552448   |  Sample_luminallactate_01 |
|SRR1552449   |  Sample_luminallactate_02 |
|SRR1552450   |  Sample_basalvirgin_01 |
|SRR1552451   |  Sample_basalvirgin_02 |
|SRR1552452   |  Sample_basalpregnant_01 |
|SRR1552453   |  Sample_basalpregnant_02 |
|SRR1552454   |  Sample_basallactate_01 |
|SRR1552455   |  Sample_basallactate_02 |



In [18]:
!mv SRR1552446.fastq.gz Sample_luminalvirgin_02.fastq.gz

Now based on what we observed in our fastqc outputs lets start doing some QC on our actual data, we will be using cutadapt which is a popular software but my no means the only one.

In [ ]:
!pip install cutadapt


In [20]:
!cutadapt --version

5.0


Lets run cutadapt on a sample, asking to cut the first 15 bases and to keep sequences with quality scores higher than 28

In [ ]:
!cutadapt --cut 15 -q 28  -o Trimmed_luminalvirgin_02.fastq Sample_luminalvirgin_02.fastq.gz

Let's run fastqc on it one more time and download the data,
what do you observe?
remember to check the actual file name to download!

In [ ]:
!./FastQC/fastqc Trimmed_luminalvirgin_02.fastq

In [ ]:
#Great quality ber base read once again
#Error is seen in seq length distribution

### 2. Read Mapping
We will be using hisat2, remember there are many options!! We will use this as a case example, right now I just want you to be aware that this is not the only option!

First, we will install hisat2

In [ ]:
!apt-get install hisat2

We are going to use the approach "align against a reference genome", from last week lecture you should remember that there were multiple posibilities, to do that we have to
1) Download the genome, 2) run hisat2 with our data

In [ ]:
!wget ftp://ftp.ccb.jhu.edu/pub/infphilo/hisat2/data/mm10.tar.gz

In [25]:
!tar -xzf mm10.tar.gz



In [26]:
!ls mm10

genome.1.ht2  genome.3.ht2  genome.5.ht2  genome.7.ht2	make_mm10.sh
genome.2.ht2  genome.4.ht2  genome.6.ht2  genome.8.ht2


In [ ]:
!hisat2 -x mm10/genome -U SRR1552444.fastq.gz -S Aln_luminalvirgin_02.sam

In [ ]:
!hisat2 -x mm10/genome -U SRR1552453.fastq.gz -S Aln_luminalvirgin_03.sam


Let's take a look at the alignment files!, take a moment to identify the read name, the sequence where it is aligning, the alignment shorthad string

In [ ]:
!samtools view Aln_luminalvirgin_02.sam


Interestingly we can run fastqc on a sam/bam file, take a moment to run it and donwload it as we have done before!, Hint: Here is the first line of code. What do you observe!

In [30]:
!./FastQC/fastqc  Aln_luminalvirgin_02.sam

Started analysis of Aln_luminalvirgin_02.sam
Approx 100% complete for Aln_luminalvirgin_02.sam
Analysis complete for Aln_luminalvirgin_02.sam


In [31]:
!ls

Aln_luminalvirgin_02_fastqc.html  SRR1552444_fastqc.zip
Aln_luminalvirgin_02_fastqc.zip   SRR1552444.fastq.gz
Aln_luminalvirgin_02.sam	  SRR1552447.fastq.gz
Aln_luminalvirgin_03.sam	  SRR1552448.fastq.gz
chr1_mm10.00.b.array		  SRR1552449.fastq.gz
chr1_mm10.00.b.tab		  SRR1552450.fastq.gz
chr1_mm10.files			  SRR1552451.fastq.gz
chr1_mm10.reads			  SRR1552452.fastq.gz
example.zip			  SRR1552453.fastq.gz
FastQC				  SRR1552454.fastq.gz
fastqc_v0.11.9.zip		  SRR1552455.fastq.gz
mm10				  targets2.txt
mm10.tar.gz			  Trimmed_luminalvirgin_02.fastq
sample_data			  Trimmed_luminalvirgin_02_fastqc.html
Sample_luminalvirgin_02.fastq.gz  Trimmed_luminalvirgin_02_fastqc.zip
SRR1552444_fastqc.html


In [ ]:
#Per base seq is off at the beginning position of each read

### 3.- Features counts
We will bse now using subread (https://subread.sourceforge.net/), which comprises a suite of software programs for processing next-gen sequencing read data, importantly for our case it can sum and summarize hits really fast

In [ ]:
!apt-get install subread

In [ ]:
!wget https://ftp.ensembl.org/pub/release-102/gtf/mus_musculus/Mus_musculus.GRCm38.102.gtf.gz

The reference genome does not have coordiates per each gene, so we will be downloading a GTF files (general feature format) You can learn more about a GTF file from your favorite website (https://useast.ensembl.org/info/website/upload/gff.html)

In [34]:
!gunzip Mus_musculus.GRCm38.102.gtf.gz

In [ ]:
!head -n 100 Mus_musculus.GRCm38.102.gtf

We ca finally count features! But before going there take a moment to run cutadapt and hisat2 on two more samples.
HINT: We are reapeaing the code boxes that start with "!cutadapt --cut" and "!hisat2 -x " do it for at two more samples, remeber to change the names and track them as trimmed_XXXXX and aln_XXXXX

1lsHopefully after this you should have three more files, run features counts on them as follow, (NOTE the code block below might not work unless

In [ ]:
!featureCounts -a Mus_musculus.GRCm38.102.gtf -o counts.txt  *.sam

In [37]:
!head -n 50 counts.txt

# Program:featureCounts v2.0.3; Command:"featureCounts" "-a" "Mus_musculus.GRCm38.102.gtf" "-o" "counts.txt" "Aln_luminalvirgin_02.sam" "Aln_luminalvirgin_03.sam" 
Geneid	Chr	Start	End	Strand	Length	Aln_luminalvirgin_02.sam	Aln_luminalvirgin_03.sam
ENSMUSG00000102693	1	3073253	3074322	+	1070	0	0
ENSMUSG00000064842	1	3102016	3102125	+	110	0	0
ENSMUSG00000051951	1;1;1;1;1;1;1	3205901;3206523;3213439;3213609;3214482;3421702;3670552	3207317;3207317;3215632;3216344;3216968;3421901;3671498	-;-;-;-;-;-;-	6094	0	0
ENSMUSG00000102851	1	3252757	3253236	+	480	0	0
ENSMUSG00000103377	1	3365731	3368549	-	2819	0	0
ENSMUSG00000104017	1	3375556	3377788	-	2233	0	0
ENSMUSG00000103025	1	3464977	3467285	-	2309	0	0
ENSMUSG00000089699	1;1	3466587;3513405	3466687;3513553	+;+	250	0	0
ENSMUSG00000103201	1	3512451	3514507	-	2057	0	0
ENSMUSG00000103147	1	3531795	3532720	+	926	0	0
ENSMUSG00000103161	1	3592892	3595903	-	3012	0	0
ENSMUSG00000102331	1;1	3647309;3658847	3650509;3658904	-;-	3259	0	0
ENSMUSG00000102348	

#### 4.- Now let's visualize our data!

Download your counts table and load it into R. Answer the following questions on an R-markdown. I want to see your code!

1.- Use an online resource and look up the function mutate and use it to to sum all the counts per row. Which is the gene with most counts accross all samples.

2.- What Is the length of the longest Gene and shortest Gene.

3.- Use ggplot() and plot a histogram of counts across all samples (your new col with the sum).

4.- use heatmap to visualize gene expression (Each row represents a gene, Each column represents a sample, The color represents the effect size)
